In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
filepath = "../Data/Combined/FHD_55.csv"
data = pd.read_csv(filepath, index_col=False)

In [3]:
#for val in data:
#    if val != 'time':
#        data[val] = (data[val] - min(data[val])) / (max(data[val]) - min(data[val]))

In [4]:
data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

In [5]:
idx_max = -1
max = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > max:
        max = float(data['controller_right_vel'][i])
        idx_max = i

In [6]:
max, idx_max

(10.075528058846395, 103)

In [7]:
data['controller_right_vel.y'][idx_max]

5.864957

In [27]:
filepath = "../Data/Combined/SRV_50.csv"
data = pd.read_csv(filepath, index_col=False)

data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)
data['controller_right_rel_headset'] = (data['headset_pos.z']) - (data['controller_right_pos.z'])


idx_max = -1
max = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > max:
        max = float(data['controller_right_vel'][i])
        idx_max = i

print(data['controller_right_vel.y'][idx_max])
print(data['controller_right_vel.x'][idx_max])
print(data['controller_left_vel.z'][idx_max])

-11.74106
0.1610769
-2.119427


In [ ]:
def simple_stat_classifier(filepath):
    data = pd.read_csv(filepath, index_col=False)
    data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

    idx_max = -1
    max = -float('inf')
    for i in range(len(data['controller_right_vel'])):
        if float(data['controller_right_vel'][i]) > max:
            max = float(data['controller_right_vel'][i])
            idx_max = i

    if float(data['controller_right_vel.y'][idx_max]) > 0:
        # BHD or FHD
        if float(data['controller_right_vel.x'][idx_max]) < 0:
            return "FHD"
        else:
            return "BHD"
    else:
        # SRV or VOL
        if float(data['controller_right_vel.y'][idx_max]) < -3.5: # Really wouldn't want to use this part...
            return "SRV"
        else:
            return "VOL"

In [ ]:
simple_stat_classifier("../Data/Combined/BHD_13.csv")

'BHD'

In [ ]:
actual = []
predicted = []
csv_number = []
for action in ['BHD','FHD','VOL','SRV']:
    for i in range(1,91):
        filepath = "../Data/Combined/" + action + "_" + str(i).zfill(2) + ".csv"
        actual.append(action)
        predicted.append(simple_stat_classifier(filepath))
        csv_number.append(i)

In [ ]:
for i in range(len(actual)):
    if actual[i] != predicted[i]:
        print(actual[i], predicted[i], csv_number[i])

BHD FHD 84
VOL SRV 13
VOL SRV 67
SRV VOL 3
SRV VOL 9
SRV BHD 11
SRV VOL 73


In [ ]:
metrics.accuracy_score(actual, predicted)

0.9805555555555555

Preliminary Data Analysis seems to show that if we just take the controller_right_vel.y datapoint at the highest controller velocity, we can differentiate between serves and FHD/BHD/VOL. 

SRV is highly negative. (controller moving downwards)

FHD, BHD, are highly positive. (controller moving upwards)

VOL is slightly negative. (controller moving downwards)

Then out of FHD, BHD, we can check controller_right_vel.x
 - If > 0, BHD. (Controller moving to the right)
 - If < 0, FHD. (Controller moving to the left)

But magnitude is dependent on person, so we might not be able to use that... Might only be able to use +/-